# Human Hand Data Inspection

This notebook inspects the processed human hand data from RealSense recordings.


In [2]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
import json

# Set up matplotlib for better display
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 12


In [3]:
# File path
hdf5_path = "Your Path"

print(f"Checking file: {hdf5_path}")
print(f"File exists: {Path(hdf5_path).exists()}")

if Path(hdf5_path).exists():
    # Open HDF5 file
    with h5py.File(hdf5_path, 'r') as f:
        print("\n=== HDF5 File Structure ===")
        
        def print_structure(name, obj):
            print(name)
        
        f.visititems(print_structure)
        
        print("\n=== File Attributes ===")
        for key in f.attrs:
            print(f"{key}: {f.attrs[key]}")
else:
    print("File not found! Please check the path.")


Checking file: Your Path
File exists: False
File not found! Please check the path.


In [4]:
if Path(hdf5_path).exists():
    with h5py.File(hdf5_path, 'r') as f:
        print("=== Data Content Analysis ===")
        
        # Check if it's in robomimic format (with data group)
        if 'data' in f:
            print("\n--- RoboMimic Format Detected ---")
            data_group = f['data']
            demo_keys = list(data_group.keys())
            print(f"Number of demos: {len(demo_keys)}")
            
            if demo_keys:
                # Inspect first demo
                first_demo = demo_keys[1]
                demo = data_group[first_demo]
                print(f"\n--- First Demo: {first_demo} ---")
                
                # Print demo structure
                def print_demo_structure(name, obj):
                    if isinstance(obj, h5py.Dataset):
                        print(f"  {name}: shape={obj.shape}, dtype={obj.dtype}")
                    else:
                        print(f"  {name}: group")
                
                demo.visititems(print_demo_structure)
                
                # Print demo attributes
                print(f"\nDemo attributes:")
                for key in demo.attrs:
                    print(f"  {key}: {demo.attrs[key]}")
        
        # Check if it's raw RealSense format
        elif 'rgb' in f:
            print("\n--- Raw RealSense Format Detected ---")
            print(f"RGB shape: {f['rgb'].shape}")
            print(f"Depth shape: {f['depth'].shape}")
            print(f"Timestamps shape: {f['timestamps'].shape}")
            
            # Print attributes
            print("\nFile attributes:")
            for key in f.attrs:
                print(f"  {key}: {f.attrs[key]}")
        
        else:
            print("\n--- Unknown Format ---")
            print("Available top-level keys:", list(f.keys()))


In [5]:
if Path(hdf5_path).exists():
    with h5py.File(hdf5_path, 'r') as f:
        if 'data' in f:
            data_group = f['data']
            demo_keys = list(data_group.keys())
            
            if demo_keys:
                # Get first demo
                first_demo = demo_keys[0]
                demo = data_group[first_demo]
                
                # Check available image types
                image_types = []
                if 'obs/front_img_1' in demo:
                    image_types.append('front_img_1')
                if 'obs/front_img_1_masked' in demo:
                    image_types.append('front_img_1_masked')
                if 'obs/front_img_1_line' in demo:
                    image_types.append('front_img_1_line')
                if 'obs/front_img_1_mask' in demo:
                    image_types.append('front_img_1_mask')
                
                print(f"Available image types: {image_types}")
                
                # Visualize different image types
                num_images = min(5, demo['obs/front_img_1'].shape[0]) if 'obs/front_img_1' in demo else 0
                
                for img_type in image_types:
                    print(f"\n=== {img_type.upper()} ===")
                    # images = demo[f'obs/{img_type}'][:num_images]
                    start_frame = 200
                    num_images = 5
                    images = demo[f'obs/{img_type}'][start_frame:start_frame + num_images]
                    print(f"Shape: {images.shape}")
                    
                    # Create subplot for this image type
                    fig, axes = plt.subplots(1, num_images, figsize=(20, 4))
                    if num_images == 1:
                        axes = [axes]
                    
                    for i in range(num_images):
                        if img_type == 'front_img_1_mask':
                            # Binary mask
                            axes[i].imshow(images[i], cmap='gray')
                        else:
                            # RGB images
                            if images[i].shape[2] == 3:
                                # Convert BGR to RGB if needed
                                # img_display = cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB)
                                img_display = images[i]
                            else:
                                img_display = images[i]
                            axes[i].imshow(img_display)
                        
                        axes[i].set_title(f'Frame {i}')
                        axes[i].axis('off')
                    
                    plt.tight_layout()
                    plt.show()
        
        else:
            print("No robomimic format data found")


In [6]:
if Path(hdf5_path).exists():
    with h5py.File(hdf5_path, 'r') as f:
        if 'rgb' in f:
            print("=== Raw RealSense Data Visualization ===")
            
            rgb_images = f['rgb'][:]
            depth_images = f['depth'][:]
            
            print(f"RGB images shape: {rgb_images.shape}")
            print(f"Depth images shape: {depth_images.shape}")
            
            # Visualize first few frames
            num_frames = min(5, len(rgb_images))
            
            fig, axes = plt.subplots(2, num_frames, figsize=(20, 8))
            if num_frames == 1:
                axes = axes.reshape(2, 1)
            
            for i in range(num_frames):
                # RGB image
                # rgb_img = cv2.cvtColor(rgb_images[i], cv2.COLOR_BGR2RGB)
                rgb_img = rgb_images[i]
                axes[0, i].imshow(rgb_img)
                axes[0, i].set_title(f'RGB Frame {i}')
                axes[0, i].axis('off')
                
                # Depth image
                depth_img = depth_images[i]
                # Normalize depth for visualization
                depth_normalized = cv2.normalize(depth_img, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
                axes[1, i].imshow(depth_normalized, cmap='jet')
                axes[1, i].set_title(f'Depth Frame {i}')
                axes[1, i].axis('off')
            
            plt.tight_layout()
            plt.show()
            
            # Print some statistics
            print(f"\n=== Data Statistics ===")
            print(f"RGB value range: [{rgb_images.min()}, {rgb_images.max()}]")
            print(f"Depth value range: [{depth_images.min()}, {depth_images.max()}]")
            print(f"Number of frames: {len(rgb_images)}")
            
            if 'timestamps' in f:
                timestamps = f['timestamps'][:]
                print(f"Timestamp range: [{timestamps[0]:.3f}, {timestamps[-1]:.3f}] seconds")
                print(f"Duration: {timestamps[-1] - timestamps[0]:.3f} seconds")
                print(f"Average FPS: {len(timestamps) / (timestamps[-1] - timestamps[0]):.2f}")


In [7]:
if Path(hdf5_path).exists():
    with h5py.File(hdf5_path, 'r') as f:
        if 'data' in f:
            data_group = f['data']
            demo_keys = list(data_group.keys())
            
            if demo_keys:
                print("=== Actions and Poses Analysis ===")
                
                # Analyze first demo
                first_demo = demo_keys[1]
                demo = data_group[first_demo]
                
                # Actions analysis
                if 'actions_xyz' in demo:
                    actions = demo['actions_xyz'][:]
                    print(f"\n--- Actions (actions_xyz) ---")
                    print(f"Shape: {actions.shape}")
                    print(f"Value range: [{actions.min():.3f}, {actions.max():.3f}]")
                    print(f"Mean: {actions.mean():.3f}, Std: {actions.std():.3f}")
                    
                    # Plot action trajectories
                    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
                    
                    # Reshape for plotting (flatten time dimension)
                    actions_flat = actions.reshape(-1, actions.shape[-1])
                    
                    if actions.shape[-1] >= 3:
                        # Plot X, Y, Z trajectories
                        axes[0, 0].plot(actions_flat[:, 0], label='X')
                        axes[0, 0].set_title('X Trajectory')
                        axes[0, 0].legend()
                        
                        axes[0, 1].plot(actions_flat[:, 1], label='Y')
                        axes[0, 1].set_title('Y Trajectory')
                        axes[0, 1].legend()
                        
                        axes[1, 0].plot(actions_flat[:, 2], label='Z')
                        axes[1, 0].set_title('Z Trajectory')
                        axes[1, 0].legend()
                        
                        # 3D trajectory
                        ax = axes[1, 1]
                        ax = plt.subplot(2, 2, 4, projection='3d')
                        ax.plot(actions_flat[:, 0], actions_flat[:, 1], actions_flat[:, 2])
                        ax.set_xlabel('X')
                        ax.set_ylabel('Y')
                        ax.set_zlabel('Z')
                        ax.set_title('3D Trajectory')
                    
                    plt.tight_layout()
                    plt.show()
                
                # EE poses analysis
                if 'obs/ee_pose' in demo:
                    ee_poses = demo['obs/ee_pose'][:]
                    print(f"\n--- End-Effector Poses (ee_pose) ---")
                    print(f"Shape: {ee_poses.shape}")
                    print(f"Value range: [{ee_poses.min():.3f}, {ee_poses.max():.3f}]")
                    print(f"Mean: {ee_poses.mean():.3f}, Std: {ee_poses.std():.3f}")
                
                # Actions_xyz_act analysis
                if 'actions_xyz_act' in demo:
                    actions_act = demo['actions_xyz_act'][:]
                    print(f"\n--- Interpolated Actions (actions_xyz_act) ---")
                    print(f"Shape: {actions_act.shape}")
                    print(f"actions_act: {actions_act[0][:10]}")
                    print(f"Value range: [{actions_act.min():.3f}, {actions_act.max():.3f}]")
                    print(f"Mean: {actions_act.mean():.3f}, Std: {actions_act.std():.3f}")


In [8]:
if Path(hdf5_path).exists():
    with h5py.File(hdf5_path, 'r') as f:
        print("=== Data Summary ===")
        
        if 'data' in f:
            data_group = f['data']
            demo_keys = list(data_group.keys())
            
            print(f"Data format: RoboMimic")
            print(f"Number of demos: {len(demo_keys)}")
            
            total_samples = 0
            for demo_key in demo_keys:
                demo = data_group[demo_key]
                if 'num_samples' in demo.attrs:
                    total_samples += demo.attrs['num_samples']
            
            print(f"Total samples: {total_samples}")
            
            # Check data consistency
            if demo_keys:
                first_demo = data_group[demo_keys[0]]
                
                print(f"\n--- Data Consistency Check ---")
                
                # Check if all required datasets exist
                required_datasets = ['actions_xyz', 'obs/front_img_1', 'obs/ee_pose']
                for dataset in required_datasets:
                    if dataset in first_demo:
                        print(f"✅ {dataset}: {first_demo[dataset].shape}")
                    else:
                        print(f"❌ {dataset}: Missing")
                
                # Check optional datasets
                optional_datasets = ['actions_xyz_act', 'obs/front_img_1_masked', 'obs/front_img_1_line', 'obs/front_img_1_mask']
                for dataset in optional_datasets:
                    if dataset in first_demo:
                        print(f"✅ {dataset}: {first_demo[dataset].shape}")
                    else:
                        print(f"⚠️  {dataset}: Not available")
        
        elif 'rgb' in f:
            print(f"Data format: Raw RealSense")
            print(f"RGB frames: {f['rgb'].shape[0]}")
            print(f"Depth frames: {f['depth'].shape[0]}")
            print(f"Resolution: {f['rgb'].shape[1]}x{f['rgb'].shape[2]}")
            
            if 'timestamps' in f:
                timestamps = f['timestamps'][:]
                duration = timestamps[-1] - timestamps[0]
                fps = len(timestamps) / duration
                print(f"Duration: {duration:.2f} seconds")
                print(f"FPS: {fps:.2f}")
        
        else:
            print("Unknown data format")
            print(f"Available keys: {list(f.keys())}")
        
        print("\n=== Inspection Complete ===")
